In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm

In [2]:
trainingDataPath = '100_celebs_dataset/training/'
testDataPath = '100_celebs_dataset/test/'
criterion = nn.CrossEntropyLoss() # includes softmax function

In [3]:
# Define the device (use GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # either 'cude' or 'cpu'
print(f'Using device: {device}')
# If it comes back as cpu, you don't have your gpu set up right to work

Using device: cuda


In [4]:
class twoLayerModel(nn.Module):
    # num_classes = number of possible labels
    # input_size = size of images
    def __init__(self, input_size, hidden_size, num_classes):
        super(twoLayerModel, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size) # Creates a linear layer given the input size and output size
        self.layer2 = nn.Linear(hidden_size, num_classes)
        self.input_size = input_size

    def forward(self, x):
        x = x.view(-1, self.input_size)  # Flatten the input tensor, since we got to input it into the layer of the nn (and they don't take matrixes)
        x = self.layer1(x) # Layer1
        x= F.relu(x) # Activation function pre-Layer 2
        return self.layer2(x) # Layer 2, we'll do activFunc in Cross Entropy Loss

In [5]:
def scale(width):
    return transforms.Compose([
    transforms.Resize((width, width)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(), # automatically normalizes data from -1 to 1
    transforms.Normalize((0.5,), (0.5,)) # further scales it to desired -0.5 and 0.5
])

In [6]:
def getScaledData(scale_width):
    trainData = datasets.ImageFolder(root=trainingDataPath, transform=scale(scale_width))
    testData = datasets.ImageFolder(root=testDataPath, transform=scale(scale_width))
    return trainData, testData

In [7]:
def train_model(model, train_loader, criterion, optimizer, epochs):
    time_start = time()
    model.train()  # Set model to training mode
    for epoch in range(epochs):
        total_loss = 0
        for images, labels in train_loader:
            images, labels = images.to('cuda'), labels.to('cuda') # Needed to make run on GPU
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}')

    time_stop = time()
    time_elapsed = time_stop - time_start
    print(f'elapsed time {round(time_elapsed,1)} sec.')

In [13]:
def test_model(model, train_loader, test_loader):
    model.eval()  # Set model to evaluation mode, just cares about output
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in train_loader:
            images, labels = images.to('cuda'), labels.to('cuda')
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    train_acc = 100 * correct / total
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to('cuda'), labels.to('cuda')
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc = 100 * correct / total
    print(f'Accuracy of the model on the training dataset: {train_acc:.2f}%')
    print(f'Accuracy of the model on the test dataset: {test_acc:.2f}%')

In [17]:
# res: will scale all images down to this width
# hidden: number of hidden layers
# classes: number of categories in the classification
# lr: learning rate
# epochs: number of epochs to train for
# batchSize: size of batches
def train_test(res, hidden, classes, lr, epochs, batchSize):
    model = twoLayerModel(res**2, hidden, classes).to('cuda')
    optimizer = optim.Adam(model.parameters(), lr=lr)
    trainDataSet, testDataSet = getScaledData(res)
    trainLoader = DataLoader(dataset=trainDataSet, batch_size=batchSize, shuffle=True, drop_last=True)
    testLoader = DataLoader(dataset=testDataSet, batch_size=batchSize, shuffle=False, drop_last=True)
    train_model(model, trainLoader, criterion, optimizer, epochs=epochs)
    test_model(model, trainLoader, testLoader)

In [18]:
train_test(50, 60, 100, 0.001, 20, 20)

Epoch [1/20], Loss: 4.4045
Epoch [2/20], Loss: 3.9291
Epoch [3/20], Loss: 3.6572
Epoch [4/20], Loss: 3.4526
Epoch [5/20], Loss: 3.2802
Epoch [6/20], Loss: 3.1139
Epoch [7/20], Loss: 2.9709
Epoch [8/20], Loss: 2.8473
Epoch [9/20], Loss: 2.7382
Epoch [10/20], Loss: 2.6155
Epoch [11/20], Loss: 2.5181
Epoch [12/20], Loss: 2.4174
Epoch [13/20], Loss: 2.3373
Epoch [14/20], Loss: 2.2631
Epoch [15/20], Loss: 2.1738
Epoch [16/20], Loss: 2.0992
Epoch [17/20], Loss: 2.0357
Epoch [18/20], Loss: 1.9682
Epoch [19/20], Loss: 1.9103
Epoch [20/20], Loss: 1.8392
elapsed time 495.8 sec.
Accuracy of the model on the training dataset: 58.64%
Accuracy of the model on the test dataset: 12.40%
